##Técnica de los K Vecinos más Cercanos

In [1]:
import time
import pandas as pd
from itertools import combinations
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, max_error, r2_score

# Cargar datos desde el archivo CSV
df_datos = pd.read_csv("datos/insurance.csv")

# Codificación One Hot para la columna 'region'
codificador_one_hot = OneHotEncoder(sparse=False)
codificacion_one_hot = codificador_one_hot.fit_transform(df_datos[['region']])
arr_nombre_nuevas_columnas = ['region_' + str(cat) for cat in codificador_one_hot.categories_[0]]
df_nuevas_columnas_one_hot = pd.DataFrame(codificacion_one_hot, columns=arr_nombre_nuevas_columnas)
df_datos = pd.concat([df_datos, df_nuevas_columnas_one_hot], axis=1)

# Columnas incluyendo 'region'
columnas_especificas = ['age', 'bmi'] + arr_nombre_nuevas_columnas + ['children', 'sex']

# Crear conjuntos de datos de entrenamiento y prueba para las columnas seleccionadas
df_X = pd.DataFrame(df_datos, columns=columnas_especificas)
df_y = pd.DataFrame(df_datos, columns=['charges'])
df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=100)

# Llenar valores faltantes con SimpleImputer
imputer = SimpleImputer(strategy='mean')

df_X_train_filled = pd.DataFrame(imputer.fit_transform(df_X_train), columns=df_X_train.columns)
df_X_test_filled = pd.DataFrame(imputer.transform(df_X_test), columns=df_X_test.columns)

# Normalizar variables independientes usando MinMaxScaler
scaler_X = MinMaxScaler()
df_X_train_scaled = pd.DataFrame(scaler_X.fit_transform(df_X_train_filled), columns=df_X_train_filled.columns)
df_X_test_scaled = pd.DataFrame(scaler_X.transform(df_X_test_filled), columns=df_X_test_filled.columns)

# Normalizar variable dependiente usando MinMaxScaler
scaler_y = MinMaxScaler()
df_y_train_scaled = pd.DataFrame(scaler_y.fit_transform(df_y_train), columns=df_y_train.columns)
df_y_test_scaled = pd.DataFrame(scaler_y.transform(df_y_test), columns=df_y_test.columns)

# Lista para almacenar los resultados
resultados_knn = []

# Valores de k para KNN
valores_k = [1, 2, 3]

# Iterar sobre los valores de k
for k in valores_k:
    # Generar todas las combinaciones posibles de columnas
    for i in range(1, len(columnas_especificas) + 1):
        combinaciones_columnas = combinations(columnas_especificas, i)
        for combo in combinaciones_columnas:
            # Convertir la combinación en una lista para poder seleccionar las columnas del DataFrame
            combo_lista = list(combo)
            # Crear y entrenar el modelo de regresión KNN
            model = KNeighborsRegressor(n_neighbors=k)
            model.fit(df_X_train_scaled[combo_lista], df_y_train_scaled.values.ravel())
            # Predicciones
            arr_y_test_predicho_scaled = model.predict(df_X_test_scaled[combo_lista])

            # Desnormalizar las predicciones usando el escalador de datos de prueba
            arr_y_test_predicho_desnormalizado = scaler_y.inverse_transform(arr_y_test_predicho_scaled.reshape(-1, 1)).flatten()

            # Desnormalizar los datos de prueba y entrenamiento
            df_y_test_desnormalizado = scaler_y.inverse_transform(df_y_test_scaled.values).flatten()
            # Desnormalizar los datos de prueba y entrenamiento
            df_X_test_desnormalizado = pd.DataFrame(scaler_X.inverse_transform(df_X_test_scaled), columns=df_X_test_scaled.columns)
            df_X_test_desnormalizado = df_X_test_desnormalizado.loc[:, combo_lista]
            
            # Métricas de evaluación del modelo
            max_ea = max_error(df_y_test_desnormalizado, arr_y_test_predicho_desnormalizado)
            mae = mean_absolute_error(df_y_test_desnormalizado, arr_y_test_predicho_desnormalizado)
            mse = mean_squared_error(df_y_test_desnormalizado, arr_y_test_predicho_desnormalizado)
            r2 = r2_score(df_y_test_desnormalizado, arr_y_test_predicho_desnormalizado)

            # Almacenar resultados en la lista
            resultados_knn.append({
                'Valor_K': k,
                'Columnas': combo_lista,
                'Máximo_EA': max_ea,
                'EA_Medio': mae,
                'R^2': r2
            })

# Crear un DataFrame a partir de la lista de resultados
df_resultados_knn = pd.DataFrame(resultados_knn)

# Imprimir resultados
for idx, row in df_resultados_knn.iterrows():
    print(f"Resultados para K={row['Valor_K']} y Columnas={row['Columnas']}:")
    print(f"   Máximo EA: {row['Máximo_EA']:.2f}")
    print(f"   EA Medio: {row['EA_Medio']:.2f}")
    print(f"   R^2: {row['R^2']:.4f}")
    print("-" * 90)

C:\Users\rimel\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Resultados para K=1 y Columnas=['age']:
   Máximo EA: 55310.88
   EA Medio: 10365.32
   R^2: -0.7796
------------------------------------------------------------------------------------------
Resultados para K=1 y Columnas=['bmi']:
   Máximo EA: 56999.59
   EA Medio: 12197.21
   R^2: -0.8657
------------------------------------------------------------------------------------------
Resultados para K=1 y Columnas=['region_northeast']:
   Máximo EA: 55301.66
   EA Medio: 10141.43
   R^2: -0.4142
------------------------------------------------------------------------------------------
Resultados para K=1 y Columnas=['region_northwest']:
   Máximo EA: 49906.39
   EA Medio: 10014.97
   R^2: -0.0392
------------------------------------------------------------------------------------------
Resultados para K=1 y Columnas=['region_southeast']:
   Máximo EA: 43906.09
   EA Medio: 10299.26
   R^2: -0.1579
------------------------------------------------------------------------------------------
R